In [ ]:
import tifffile
from griottes import generate_contact_graph, generate_geometric_graph, plot_2D


# Load image and mask of nuclei

First download the image from the latest release of `Griottes`.

In [ ]:
import urllib3
import shutil
import os

url = 'https://github.com/BaroudLab/Griottes/releases/download/v1.0-alpha/zebrafish_brain_cell_labels.tiff'
filename = 'zebrafish_brain_cell_labels.tiff'

if not os.path.exists(filename):
    c = urllib3.PoolManager()

    with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
        shutil.copyfileobj(resp, out_file)

    resp.release_conn()
else:
    print('dataset already exists')

We then load the image to generate the different graphs from it.

In [ ]:
image = tifffile.imread(filename)
image.shape

# Generating a contact graph

From the masks of the complete cells we can build the contact graph of the network, connecting together cells sharing a common border.

In [ ]:
G_contact = generate_contact_graph(image[..., 4],
                                                 analyze_fluo_channels=False, 
                                                 radius = 100)

In [4]:
G_geometric = generate_geometric_graph(image[..., 4],
                                                 analyze_fluo_channels=False,
                                                 cell_cell_distance = 200,
                                                 radius = 100)

## Plotting a representation of the graph

From the graph `G` it is possible to plot a visual representation of the network. NetworkX has several plotting functions available for representing graphs, however `Griottes` contains several specific plotting functions adapted for the network representation of tisssues. These functions are called through the `graph_plot` module.

In [ ]:
plot_2D(G_geometric,
                background_image=image[...,1],
                figsize = (5,5),
                alpha_line = 1,
                scatterpoint_size = 18,
                edge_color = 'w',
                line_factor = 1)

Adding the `background_image` variable allows to plot the network on to the tissue image of your choice. Here we do it with the contact graph:

In [ ]:
graph_plot.network_plot_2D(G_contact,
                background_image = image[:, :,1],
                figsize = (5,5),
                alpha_line = 1,
                scatterpoint_size = 18,
                legend = True,
                weights = False,
                edge_color = 'w',
                line_factor = 0.15)

We can use the information contained within the masks to extract the shared border size. This is reflected within the network by weighted links. The thicker the edge connecting two nodes, the bigger the shared membrane. To add variable weights, just change the `weights` to `True`.

In [ ]:
plot_2D(G_contact,
                background_image = image[:, :,1],
                figsize = (5,5),
                alpha_line = .5,
                scatterpoint_size = 10,
                legend = True,
                edge_color = 'w',
                line_factor = 0.15)